In [ ]:
import os
import image2coordinate as i2c
import numpy as np

In [ ]:
# dataset_dir = '/Users/hsyoon/Downloads/dataset2/'
# print(os.listdir(dataset_dir))
# for dir in os.listdir(dataset_dir):
#     path = os.path.join(dataset_dir, dir)
#     print(path)
#     i2c.process_images_in_folder(path)

dataset_dir = '/Users/hsyoon/Downloads/dataset2/26'
i2c.process_images_in_folder(dataset_dir)


In [ ]:
# csv shape 확인용 (image개수, 63)
csvs = '/Users/hsyoon/Downloads/dataset2/csv/'

for csv in sorted(os.listdir(csvs)):
    csv_path = os.path.join(csvs,csv)
    if csv_path.endswith("csv"):
        a = np.loadtxt(csv_path, delimiter=",")
        print(csv, ":", a.shape)

In [58]:
from tensorflow.keras import models, layers
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt


def load_data(csvs_dir):
    train = np.empty((0, 64))
    test = np.empty((0, 64))
    for csv_name in sorted(os.listdir(csvs_dir)):
        if csv_name.endswith('csv'):
            csv_path = os.path.join(csvs_dir,csv_name)
            class_name = csv_path.split('/')[-1].split('_')[0]
            class_name = int(class_name.lstrip('0'))
            # print(class_name)   # 1~26

            val = np.loadtxt(csv_path,delimiter=',')
            class_value = np.full((val.shape[0], ), class_name) # label
            val = np.column_stack((val, class_value))
            # print(val.shape)
            train_ = val[:int(len(val)*0.9), :]
            test_ = val[int(len(val)*0.9):, :]
            # print(train_.shape)
            # print(test_.shape)
            train = np.concatenate((train, train_), axis=0)
            test = np.concatenate((test, test_), axis=0)
    train = np.random.permutation(train)     
    test = np.random.permutation(test)
    train_data = train[:, :63]
    train_labels = train[:, -1]
    test_data = test[:, :63]
    test_labels = test[:, -1]
    return (train_data, train_labels), (test_data, test_labels)

def plot_acc_loss(h):
    plt.figure(figsize=(15.6, 4.8), dpi=100)
    plt.subplot(1,2,1)
    plt.plot(h.history['accuracy'])
    plt.plot(h.history['val_accuracy'])
    plt.title('Accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Training', 'Validation'], loc=0)
    
    plt.subplot(1,2,2)
    plt.plot(h.history['loss'])
    plt.plot(h.history['val_loss'])
    plt.title('Loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Training', 'Validation'], loc=0)
    plt.show()

def build_model():
    input_shape = (63,)
    model = models.Sequential()
    model.add(layers.Dense(2560, activation='relu', input_shape=input_shape))
    model.add(layers.Dense(1280, activation='relu'))
    model.add(layers.Dense(640, activation='relu'))
    model.add(layers.Dense(26, activation='softmax'))
    model.compile(optimizer='adam',
                loss='categorical_crossentropy',
                metrics=['accuracy'])

    return model


if __name__ == '__main__':
    (train_data, train_labels), (test_data, test_labels) = load_data('/Users/hsyoon/Downloads/dataset2/csv')
    train_data.reshape(-1, 63)
    train_labels = to_categorical(train_labels-1)
    test_labels = to_categorical(test_labels-1)
    
    model = build_model()
    history = model.fit(train_data, train_labels, epochs=30, verbose=2)
    acc, loss = model.evaluate(test_data, test_labels)
    plot_acc_loss(history)

(5940, 63)
(5940, 63)
(5940, 63)
